In [1]:
from langchain_groq import ChatGroq
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
import gradio as gr

load_dotenv()
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7
)

response_schemas = [
    ResponseSchema(name="name", description="Full name of the scientist", type="string"),
    ResponseSchema(name="field", description="Primary field of research or study", type="string"),
    ResponseSchema(name="notable_works", description="List of notable scientific works or contributions", type="list"),
    ResponseSchema(name="impact", description="Brief summary of the scientist’s impact on the field", type="string")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
You are a scientific profile generator. Given the name of a scientist, return their professional profile in structured JSON format.

{format_instructions}

Examples:
- Input: Albert Einstein
  Output: {{
    "name": "Albert Einstein",
    "field": "Theoretical Physics",
    "notable_works": ["Theory of Relativity", "Photoelectric Effect", "E=mc^2"],
    "impact": "Transformed theoretical physics and introduced concepts that shaped modern science."
  }}

- Input: Marie Curie
  Output: {{
    "name": "Marie Curie",
    "field": "Chemistry and Physics",
    "notable_works": ["Discovery of Radium", "Discovery of Polonium", "Research on radioactivity"],
    "impact": "Pioneered radioactivity research, won two Nobel Prizes, and advanced medical and nuclear science."
  }}

Scientist: {scientist}
""",
    input_variables=["scientist"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | llm | output_parser

def generate_profile(scientist_name):
    if not isinstance(scientist_name, str) or not scientist_name.strip():
        return {"name": "error", "field": "", "notable_works": [], "impact": "No name provided."}
    result = chain.invoke({"scientist": scientist_name})
    return result

gr.Interface(
    fn=generate_profile,
    inputs=gr.Textbox(label="Enter Scientist Name"),
    outputs=gr.JSON(label="Generated Scientific Profile"),
    title="Scientific Profile Generator",
    description="Enter the name of a scientist to generate a structured profile using LLaMA 3 and LangChain via Groq."
).launch()

c:\Users\MITHUN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
